In [19]:
import pandas as pd
import numpy as np
from scipy.ndimage.interpolation import shift

In [20]:
def updownlimit(df_nowprice_pct,df_weight):
    '''
    INPUT:
    df_nowprice_pct:为当前时间点的价格相较于昨日收盘价的涨跌幅，如现在为10:30，则为10:30_price - precloseprice
    df_weight：当日交易的目标权重，如今日想要半仓买入某只股票，则当日的weight为0.5，做空反之
    OUTPUT:
    df_weight_final 考虑了当前时间点由于涨跌停限制买不进去情况下的最终权重，其中不能换仓的资产延续上一个交易日的权重
    '''
    # TODO 写成 numpy
    # TODO 类型检验
    if df_nowprice_pct.shape != df_weight.shape:
        raise TypeError("input data must has same dims.")
    df_islimit = (~((abs(df_nowprice_pct) > 0.097) or ppp)).astype("float32").replace(0.,np.nan) # 当天涨跌停为nan，其它为1
    df_isupdown = ((abs(df_nowprice_pct) > 0.097)).astype("float32").replace(0.,np.nan) # 当天涨跌停为1，其它为nan
    df_isupdown_weight = df_isupdown.shift(-1)*df_weight              # 涨跌停前一天的权值

    df_weight_outlimit = (df_islimit*df_weight).fillna(0)                                  #原始权值抛去涨跌停的权值

    df_weight_limitday = ((df_isupdown_weight*df_islimit).fillna(1)*df_islimit).ffill().replace(1.,np.nan) - (df_islimit*df_weight).fillna(0)# 涨跌停当天的权值,按照前一个交易日的权重进行填充

    sr_weight_sum_perday = abs(df_weight_outlimit).sum(1)
    df_weight_outlimit = df_weight_outlimit.div(sr_weight_sum_perday.values,axis=0)
    df_weight_outlimit = df_weight_outlimit.mul(1-abs(df_weight_limitday).sum(1),axis=0)#重新核算抛弃涨跌停权重后的各只股票的权重

    df_weight_final = df_weight_outlimit + df_weight_limitday.fillna(0)
    return df_weight_final

In [33]:
'''
numpy version
'''
def updownlimit(df_nowprice_pct,df_weight):
    '''
    INPUT:
    df_nowprice_pct:为当前时间点的价格相较于昨日收盘价的涨跌幅，如现在为10:30，则为10:30_price - precloseprice
    df_weight：当日交易的目标权重，如今日想要半仓买入某只股票，则当日的weight为0.5，做空反之
    OUTPUT:
    df_weight_final 考虑了当前时间点由于涨跌停限制买不进去情况下的最终权重，其中不能换仓的资产延续上一个交易日的权重
    '''
    # TODO 写成 numpy
    # TODO 类型检验
    if df_nowprice_pct.shape != df_weight.shape:
        raise TypeError("input data must has same dims.")
    df_islimit = (~(abs(df_nowprice_pct) > 0.097)).astype("float32") # 当天涨跌停为nan，其它为1
    df_islimit[df_islimit==0] = np.nan
    df_isupdown = ((abs(df_nowprice_pct) > 0.097)).astype("float32") # 当天涨跌停为1，其它为nan
    df_isupdown[df_isupdown==0] = np.nan
    df_isupdown_weight = df_isupdown.shift(-1)*df_weight              # 涨跌停前一天的权值

    df_weight_outlimit = (df_islimit*df_weight).fillna(0)                                  #原始权值抛去涨跌停的权值

    df_weight_limitday = ((df_isupdown_weight*df_islimit).fillna(1)*df_islimit).ffill().replace(1.,np.nan) - (df_islimit*df_weight).fillna(0)# 涨跌停当天的权值,按照前一个交易日的权重进行填充

    sr_weight_sum_perday = abs(df_weight_outlimit).sum(1)
    df_weight_outlimit = df_weight_outlimit.div(sr_weight_sum_perday.values,axis=0)
    df_weight_outlimit = df_weight_outlimit.mul(1-abs(df_weight_limitday).sum(1),axis=0)#重新核算抛弃涨跌停权重后的各只股票的权重

    df_weight_final = df_weight_outlimit + df_weight_limitday.fillna(0)
    return df_weight_final

In [25]:
df_price_pct = pd.DataFrame([[0.01,0.01,0.01]
                             ,[0.05,0.1,0.01]
                             ,[0.01,0.01,0.01]
                             ,[0.01,0.1,-0.099]
                             ,[np.nan,0.01,0.01]
                            ,[np.nan,0.1,0.01]
                            ,[0.01,0.01,0.01]],columns=['A','B','C'])
df_weight = pd.DataFrame([[0.1,0.2,0.7]
                          ,[0.2,-0.1,0.7]
                          ,[0.6,np.nan,0.1]
                          ,[-0.3,0.3,-0.4]
                          ,[0.25,0.5,-0.25]
                         ,[0.5,np.nan,-0.5]
                         ,[0.5,np.nan,-0.5]],columns=['A','B','C'])
print df_price_pct
print df_weight

      A     B      C
0  0.01  0.01  0.010
1  0.05  0.10  0.010
2  0.01  0.01  0.010
3  0.01  0.10 -0.099
4   NaN  0.01  0.010
5   NaN  0.10  0.010
6  0.01  0.01  0.010
      A    B     C
0  0.10  0.2  0.70
1  0.20 -0.1  0.70
2  0.60  NaN  0.10
3 -0.30  0.3 -0.40
4  0.25  0.5 -0.25
5  0.50  NaN -0.50
6  0.50  NaN -0.50


In [26]:
updownlimit(df_price_pct,df_weight)

,A,B,C
0,0.100000,0.2,0.700000
1,0.177778,0.2,0.622222
2,0.857143,0.0,0.142857
3,-0.900000,0.0,0.100000
4,0.250000,0.5,-0.250000
5,0.250000,0.5,-0.250000
6,0.500000,0.0,-0.500000


In [7]:
df = updownlimit(df_price_pct,df_weight)
print abs(df).sum(1)

NameError: name 'updownlimit' is not defined

In [11]:
df_isnotlimit = (~(abs(df_price_pct) > 0.097)).astype("float32").replace(0.,np.nan) # 当天涨跌停为nan，其它为1
print df_isnotlimit

     A    B    C
0  1.0  1.0  1.0
1  1.0  1.0  1.0
2  1.0  NaN  1.0
3  1.0  NaN  1.0


In [12]:
df_islimit = ((abs(df_price_pct) > 0.097)).astype("float32").replace(0.,np.nan) # 当天涨跌停为1，其它为nan
print df_islimit

    A    B   C
0 NaN  NaN NaN
1 NaN  NaN NaN
2 NaN  1.0 NaN
3 NaN  1.0 NaN


In [13]:
df_isupdown_weight = df_islimit.shift(-1)*df_weight              # 涨跌停前一天的权值
print df_isupdown_weight

    A    B   C
0 NaN  NaN NaN
1 NaN  0.1 NaN
2 NaN  0.3 NaN
3 NaN  NaN NaN


In [24]:
df_weight_outlimit = (df_isnotlimit*df_weight).fillna(0)                                  #原始权值抛去涨跌停的权值
abs(df_weight_outlimit).sum(1)

print df_weight_outlimit

     A    B    C
0  0.1  0.2  0.7
1  0.2  0.1  0.7
2  0.6  0.0  0.1
3  0.3  0.0  0.4


In [15]:
df_weight_limitday = ((df_isupdown_weight*df_isnotlimit).fillna(1)*df_isnotlimit).ffill().replace(1.,np.nan) - (df_isnotlimit*df_weight).fillna(0)#替换完成，最终的涨跌停当天的权值
print df_weight_limitday 

    A    B   C
0 NaN  NaN NaN
1 NaN  0.0 NaN
2 NaN  0.1 NaN
3 NaN  0.1 NaN


In [19]:
1-abs(df_weight_limitday).sum(1)

0    1.0
1    1.0
2    0.9
3    0.9
dtype: float64

In [25]:
abs(df_weight_outlimit).sum(1)

0    1.0
1    1.0
2    0.7
3    0.7
dtype: float64

In [29]:
# sr_weight_sum_perday = 1-abs(df_weight_limitday).sum(1)
sr_weight_sum_perday = abs(df_weight_outlimit).sum(1)
df_weight_outlimit = df_weight_outlimit.div(sr_weight_sum_perday.values,axis=0)
df_weight_outlimit = df_weight_outlimit.mul(1-abs(df_weight_limitday).sum(1),axis=0)
print df_weight_outlimit

          A    B         C
0  0.100000  0.2  0.700000
1  0.200000  0.1  0.700000
2  0.771429  0.0  0.128571
3  0.385714  0.0  0.514286


In [30]:
df_weight_final = df_weight_outlimit + df_weight_limitday.fillna(0)
print df_weight_final

          A    B         C
0  0.100000  0.2  0.700000
1  0.200000  0.1  0.700000
2  0.771429  0.1  0.128571
3  0.385714  0.1  0.514286


In [31]:
df_weight_final.sum(1)

0    1.0
1    1.0
2    1.0
3    1.0
dtype: float64

In [51]:
updownlimit(df_price,df_weight)

,A,B,C
0,0.1,0.2,0.7
1,0.2,0.2,0.7
2,0.6,0.2,0.1
3,0.3,0.2,0.4
